##### Copyright 2025 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Multimodal Live API - Quickstart

### This notebook has been moved in the [quickstarts](../../quickstarts) section where you can find all the guides and tutorials to get to know the Gemini capabilities. Check the [websockets](../../quickstarts/websockets/) for all websockets examples.

## Click **[here](../../quickstarts/websockets/LiveAPI_streaming_in_colab.ipynb)** to open the latest version.